# Attack Exploring

Exploring the Attack and setting the right Parameters

## Imports

### GPU Information

The following GPU is used while Training the Model. Google Colab is used for Training so the GPU might change while running it later.

**GPU-Specifications:**
*   Name: Tesla P100-PCIE
*   GPU Memory: 16280MiB

In [1]:
!nvidia-smi

Tue Aug 24 16:58:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Importing Data
Files are uploaded in Google Drive. Notebook is connected to Google Drive

In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Modulation-Classification/RadioML-Dataset/Data/CW-Attack

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/Modulation-Classification/RadioML-Dataset/Data/CW-Attack


### Importing Libraries

Cleverhans is used for generating Adversarial Examples.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as pimg
import seaborn as sns
import scipy.io
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score

# Tensorflow Libraries
!pip3 install tensorflow
!pip3 install keras-tuner==1.0.2
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()
from tensorflow.keras.layers import Dense, Flatten, ReLU, Conv2D, Input, MaxPooling2D, BatchNormalization, AveragePooling2D, Reshape, ZeroPadding2D, Dropout, Activation, Permute, Concatenate
from tensorflow.keras.models import Model,Sequential

# ART Libraries
"""
!pip3 install adversarial-robustness-toolbox[tensorflow]
from art.attacks.evasion import CarliniL2Method
from art.estimators.classification import KerasClassifier
"""

# Cleverhans Libraries
!pip3 install cleverhans
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
from carlini_wagner_l2 import CarliniWagnerL2

### Importing Model

In [4]:
# Model
Robust_CNN = tf.keras.models.load_model("../Model/Clean_Robust_CNN.h5")

### Importing Dataset

Importing Data

In [5]:
# Loading Training Data
Clean_X_Train = np.load("../Clean-Data/Clean_X_Train.npy")
Clean_y_Train = np.load("../Clean-Data/Clean_y_Train.npy")

# Loading Validation Data
f = open("../Clean-Data/Clean_X_Valid.pkl","rb")
Clean_X_Valid = pickle.load(f)

f = open("../Clean-Data/Clean_y_Valid.pkl","rb")
Clean_y_Valid = pickle.load(f)

Processing Data

In [6]:
Clean_X_Train = np.expand_dims(np.transpose(Clean_X_Train, (0,2,1)),axis=-1)

for snr in Clean_X_Valid.keys():
    Clean_X_Valid[snr] = np.expand_dims(np.transpose(Clean_X_Valid[snr], (0,2,1)),axis=-1)

## Designing Attack

Modulation Schemes

In [7]:
f = open("../Dataset/ModulationMap.pkl", "rb")
ModulationMap = pickle.load(f)
ModulationMap = dict((v,k) for k,v in ModulationMap.items())
ModulationMap

{'8PSK': 0,
 'AM-DSB': 1,
 'AM-SSB': 2,
 'BPSK': 3,
 'CPFSK': 4,
 'GFSK': 5,
 'PAM4': 6,
 'QAM16': 7,
 'QAM64': 8,
 'QPSK': 9,
 'WBFM': 10}

Attack Settings

In [8]:
# Parameters of CW
nb_Iterations = 500
Confidence = 0.9
Batch_Size = 200
Learning_Rate = 0.01
Max = np.max(Clean_X_Train)
Min = np.min(Clean_X_Train)

Exploring Parameters

In [9]:
Robust_CNN.evaluate(Clean_X_Train, Clean_y_Train)

5500/5500 [==============================] - 27s 2ms/step - loss: 1.0724 - accuracy: 0.6074


[1.072371244430542, 0.6073636412620544]

CW Attack

In [10]:
for i in range(0,Clean_X_Train.shape[0],16000):

    Data = Clean_X_Train[i+16000-Batch_Size:i+16000]
    y_True = np.argmax(Clean_y_Train[i+16000-Batch_Size:i+16000], axis=1)
    
    CW = CarliniWagnerL2(
            model_fn = Robust_CNN,
            y = y_True,
            targeted = False,
            batch_size = Batch_Size,
            clip_max = Max,
            clip_min = Min,
            max_iterations = nb_Iterations,
            confidence = Confidence,
            learning_rate = Learning_Rate
        )

    Adv = CW.attack(Data)
    
    y_Pred = np.argmax(Robust_CNN.predict(Clean_X_Train[i+16000-Batch_Size:i+16000]), axis=1)
    y_Adv_Pred = np.argmax(Robust_CNN.predict(Adv), axis=1)

    print ("i = " + str(i) +  str(", True Labels = ") + str(y_True) + str(", Predicted Adv Labels = ") + str(y_Adv_Pred) + str(", Accuracy of Attack = ") + str(1-accuracy_score(y_True,y_Adv_Pred)))

i = 0, True Labels = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], Predicted Adv Labels = [8 8 9 7 9 4 8 8 8 9 9 9 8 9 8 2 9 8 8 8 8 8 7 7 8 5 8 9 9 8 8 4 8 8 9 9 9
 8 9 8 8 8 2 8 5 8 8 8 9 8 8 8 9 8 9 8 8 8 9 5 9 9 8 7 8 9 8 9 8 8 8 8 8 8
 9 9 9 8 8 7 7 8 4 8 9 4 8 4 8 8 8 8 8 8 8 8 7 6 9 7 9 9 2 8 8 8 9 9 8 4 8
 9 8 8 9 8 8 8 8 7 8 8 8 7 8 8 9 8 8 8 8 8 7 9 8 9 8 9 9 7 9 8 7 7 8 8 7 7
 8 8 9 9 8 8 8 7 8 8 8 7 8 8 8 8 7 8 8 8 8 7 9 7 7 8 8 8 8 9 8 8 8 7 9 9 8
 9 7 8 8 9 8 4 4 9 8 8 8 8 9 8], Accuracy of Attack = 1.0
i = 16000, True Labels = [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 

In [11]:
for i in range(0,Clean_X_Train.shape[0],16000):

    Data = Clean_X_Train[i:i+Batch_Size]
    y_True = np.argmax(Clean_y_Train[i:i+Batch_Size], axis=1)
    
    CW = CarliniWagnerL2(
            model_fn = Robust_CNN,
            y = y_True,
            targeted = False,
            batch_size = Batch_Size,
            clip_max = Max,
            clip_min = Min,
            max_iterations = nb_Iterations,
            confidence = Confidence,
            learning_rate = Learning_Rate
        )

    Adv = CW.attack(Data)
    
    y_Pred = np.argmax(Robust_CNN.predict(Clean_X_Train[i:i+Batch_Size]), axis=1)
    y_Adv_Pred = np.argmax(Robust_CNN.predict(Adv), axis=1)

    print ("i = " + str(i) +  str(", True Labels = ") + str(y_True) + str(", Predicted Adv Labels = ") + str(y_Adv_Pred) + str(", Accuracy of Attack = ") + str(1-accuracy_score(y_True,y_Adv_Pred)))

i = 0, True Labels = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], Predicted Adv Labels = [ 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  1  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2 10  2  2  2  2
  2  2  2  2  2  1  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  1  2  2  2  2  2  2  2  2  2  1  2  2 10  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2 

In [12]:
for i in range(0,Clean_X_Train.shape[0],16000):

    Data = Clean_X_Train[i:i+Batch_Size]
    y_True = np.argmax(Clean_y_Train[i+7200:i+7200+Batch_Size], axis=1)
    
    CW = CarliniWagnerL2(
            model_fn = Robust_CNN,
            y = y_True,
            targeted = False,
            batch_size = Batch_Size,
            clip_max = Max,
            clip_min = Min,
            max_iterations = nb_Iterations,
            confidence = Confidence,
            learning_rate = Learning_Rate
        )

    Adv = CW.attack(Data)
    
    y_Pred = np.argmax(Robust_CNN.predict(Clean_X_Train[i+7200:i+7200+Batch_Size]), axis=1)
    y_Adv_Pred = np.argmax(Robust_CNN.predict(Adv), axis=1)

    print ("i = " + str(i) +  str(", True Labels = ") + str(y_True) + str(", Predicted Adv Labels = ") + str(y_Adv_Pred) + str(", Accuracy of Attack = ") + str(1-accuracy_score(y_True,y_Adv_Pred)))

i = 0, True Labels = [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], Predicted Adv Labels = [ 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  1  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2 10  2  2  2  2
  2  2  2  2  2  1  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  1  2  2  2  2  2  2  2  2  2  1  2  2 10  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2 